In [97]:
import cv2,os
import sys
import tifffile as tiff
import numpy as np
from sklearn.model_selection import KFold as KF


In [17]:
def facedetection(filename):
    face_cascade = cv2.CascadeClassifier('C:\\Users\\Shrey Malvi\\Downloads\\opencv\\build\\etc\\haarcascades\\haarcascade_frontalface_default.xml')
    img1=tiff.imread('fer2013/jaffe/'+filename)
    faces = face_cascade.detectMultiScale(img1, 1.3, 5)
    index=0
    for (x,y,w,h) in faces:
        cropped = img1[y :y +  h , x : x + w]
        return cropped

In [29]:
def featureextraction(scaled):
    y,x=scaled.shape
    gx=np.zeros((x,y))
    scaled = np.lib.pad(scaled, 1, 'constant', constant_values=0)
    gy = np.zeros((x, y))
    for i in range(y):
        a = np.convolve(scaled[i - 1, :], [1, 0, -1], 'valid')
        b = np.convolve(scaled[i, :], [2, 0, -2], 'valid')
        c = np.convolve(scaled[i + 1, :], [1, 0, -1], 'valid')
        gx[i, :] = np.sum([a, b, c], axis=0)
    for j in range(x):
        a = np.convolve(scaled[:, j - 1], [1, 0, -1], 'valid')
        b = np.convolve(scaled[:, j], [2, 0, -2], 'valid')
        c = np.convolve(scaled[:, j + 1], [1, 0, -1], 'valid')
        gy[:, j] = np.sum([a, b, c], axis=0)
    mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    angle=angle%180;
    for i in range(0,x):
        for j in range(0,y):
            if(angle[i,j]<=20):
                angle[i,j]=0
            elif(angle[i,j]<=40):
                angle[i,j]=1
            elif(angle[i,j]<=60):
                angle[i,j]=2
            elif(angle[i,j]<=80):
                angle[i,j]=3
            elif(angle[i,j]<=100):
                angle[i,j]=4
            elif(angle[i,j]<=120):
                angle[i,j]=5
            elif(angle[i,j]<=140):
                angle[i,j]=6
            elif(angle[i,j]<=160):
                angle[i,j]=7
            elif(angle[i,j]<=180):
                angle[i,j]=8
    angle=np.int64(angle)
    mag=np.float64(mag)
    bin_n=9
    cells=8
    blocks=2
    width = scaled.shape[0]
    height = scaled.shape[1]
    angle=np.int64(angle)
    mag=np.float64(mag)
    epsilon = sys.float_info.epsilon
    x=56
    y=56
    yd= np.int64(((height - (cells * blocks)) / cells) + 1)
    xd= np.int64(((width - (cells * blocks)) / cells) + 1)
    c=0
    histograms = []
    for i in range(0,xd,1):
        for j in range(0,yd,1):
            b_mag=mag[i*cells:i*cells+cells*blocks,j*cells:j*cells+cells*blocks]
            b_angle=angle[i*cells:i*cells+cells*blocks,j*cells:j*cells+cells*blocks]
            tempHists = []
            sumHists = np.zeros((9,))
            for m in range(0,blocks*cells,cells):
                for n in range(0,blocks*cells,cells):
                    c_mag=b_mag[m:m+cells,n:n+cells]
                    c_angle=b_angle[m:m+cells,n:n+cells]
                    tempHists.append(np.bincount(c_angle.ravel(), c_mag.ravel(), bin_n))
                    sumHists = np.sum([sumHists, np.bincount(c_angle.ravel(),c_mag.ravel(), bin_n)], axis=0)
            sumHists = np.sum(sumHists)
            for hist in tempHists:
                histograms.append(np.divide(hist, np.sqrt(np.add(np.square(sumHists), np.square(epsilon)))))        
    hist = np.hstack(histograms)
    return hist


In [123]:
def getlabel(x):
    if(x[0:2]=='NE'): 
        return 0
    if(x[0:2]=='FE'):
        return 1
    if(x[0:2]=='DI'):
        return 2
    if(x[0:2]=='SA'):
        return 3
    if(x[0:2]=='HA'):
        return 4
    if(x[0:2]=='SU'):
        return 5
    if(x[0:2]=='AN'):
        return 6
img_data_list,label_temp=[],[] 
c=0
for filename in os.listdir('fer2013/jaffe'):
    c=c+1
    temp1,label,num,temp=filename.split(".")
    
    input_img=facedetection(filename)
    input_img_resize=cv2.resize(input_img,(56,56))
    input_fx=featureextraction(input_img_resize)
    img_data_list.append(input_fx)
    label_temp.append(getlabel(label))
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
label=np.array(label_temp)

In [124]:
print(img_data.shape)
kf = KF(n_splits=6, shuffle=True).split(img_data)

train_indices, test_indices =  next(kf)
x_train = img_data[train_indices]
y_train = label[train_indices]

x_test = img_data[test_indices]
y_test = label[test_indices]
print(x_test.shape)
print(y_test.shape)

(213, 1296)
(36, 1296)
(36,)


In [94]:
print(label)
label=label.reshape((213,1))

[[6]
 [6]
 [6]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [4]
 [4]
 [4]
 [4]
 [0]
 [0]
 [0]
 [3]
 [3]
 [3]
 [5]
 [5]
 [5]
 [6]
 [6]
 [6]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [4]
 [4]
 [4]
 [0]
 [0]
 [0]
 [3]
 [3]
 [3]
 [5]
 [5]
 [5]
 [6]
 [6]
 [6]
 [2]
 [2]
 [1]
 [1]
 [1]
 [4]
 [4]
 [4]
 [4]
 [0]
 [0]
 [0]
 [3]
 [3]
 [3]
 [3]
 [5]
 [5]
 [5]
 [6]
 [6]
 [6]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [4]
 [4]
 [0]
 [0]
 [0]
 [3]
 [3]
 [3]
 [5]
 [5]
 [5]
 [6]
 [6]
 [6]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [4]
 [4]
 [4]
 [0]
 [0]
 [0]
 [3]
 [3]
 [3]
 [5]
 [5]
 [5]
 [6]
 [6]
 [6]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [4]
 [4]
 [4]
 [0]
 [0]
 [0]
 [3]
 [3]
 [3]
 [5]
 [5]
 [5]
 [6]
 [6]
 [6]
 [2]
 [2]
 [1]
 [1]
 [1]
 [4]
 [4]
 [4]
 [0]
 [0]
 [0]
 [3]
 [3]
 [3]
 [5]
 [5]
 [5]
 [6]
 [6]
 [6]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [4]
 [4]
 [4]
 [0]
 [0]
 [0]
 [3]
 [3]
 [3]
 [5]
 [5]
 [5]
 [6]
 [6]
 [6]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [4]
 [4]
 [4]
 [0]
 [0]
 [0]
 [3]
 [3]
 [3]
 [5]
 [5]
 [5]
 [6]
 [6]
 [6]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]


In [125]:
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(x_train, y_train)

#Predict the response for test dataset


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [126]:
y_pred = clf.predict(x_test)

In [127]:
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.4444444444444444


In [134]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
mlp = MLPClassifier(hidden_layer_sizes=(128,64,32),activation='relu',learning_rate_init=0.001,max_iter=200,batch_size=75)
mlp.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size=75, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(128, 64, 32), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [135]:
predictions = mlp.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[11  0  0  0  0  0  0]
 [ 0  6  0  0  0  0  0]
 [ 0  0  2  0  0  0  0]
 [ 1  1  0  2  1  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  1  6  0]
 [ 0  0  1  0  0  0  4]]
             precision    recall  f1-score   support

          0       0.92      1.00      0.96        11
          1       0.86      1.00      0.92         6
          2       0.67      1.00      0.80         2
          3       1.00      0.40      0.57         5
          4       0.00      0.00      0.00         0
          5       1.00      0.86      0.92         7
          6       1.00      0.80      0.89         5

avg / total       0.93      0.86      0.87        36



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
